# Import thư viện

In [25]:
import pandas as pd
import random
import numpy as np
from collections import Counter,defaultdict

# Load dữ liệu

In [26]:
df = pd.read_csv('./dataset/letter-recognition.data')

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19999 entries, 0 to 19998
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   T       19999 non-null  object
 1   2       19999 non-null  int64 
 2   8       19999 non-null  int64 
 3   3       19999 non-null  int64 
 4   5       19999 non-null  int64 
 5   1       19999 non-null  int64 
 6   8.1     19999 non-null  int64 
 7   13      19999 non-null  int64 
 8   0       19999 non-null  int64 
 9   6       19999 non-null  int64 
 10  6.1     19999 non-null  int64 
 11  10      19999 non-null  int64 
 12  8.2     19999 non-null  int64 
 13  0.1     19999 non-null  int64 
 14  8.3     19999 non-null  int64 
 15  0.2     19999 non-null  int64 
 16  8.4     19999 non-null  int64 
dtypes: int64(16), object(1)
memory usage: 2.6+ MB


# Chuẩn bị dữ liệu:

- x gồm các đặc trưng có trong dataset

- y là biến mục tiêu

In [28]:
x = df.iloc[:,1:].values #Lấy các cột giá trị đặc trưng trừ cột đầu tiên vì là biến mục tiêu (Các chữ cái)
y = df.iloc[:,0].values #Ngược lại chỉ lấy cột đầu tiên chính là target

# Chia dữ liệu

In [29]:
def train_test_split(x,y,test_size=0.2):
    # Ép kiểu sang numpy array để dùng được index mảng
    x = np.array(x)
    y = np.array(y)
    np.random.seed(42)
    indices = np.random.permutation(len(x)) #Tạo danh sách có index từ 0 -> N nhưng được random vị trí
    test_count = int(len(x)*test_size) #Lấy x dòng cho test
    test_index = indices[:test_count] #lấy x số đầu tiên trong danh sách đã xáo trộn
    train_index = indices[test_count:] #Lấy phần còn lại
    return x[train_index],x[test_index],y[train_index],y[test_index] 

x_train, x_test, y_train, y_test = train_test_split(x,y)

# Tính toán

## Hàm tính khoảng cách Euclid

In [30]:
def euclid_distance(x1,x2):
    return np.sqrt(np.sum((x1-x2)**2))

## Thuật toán K nearest neighbor

In [ ]:
def knn(x_train,x_test,y_train,k):
    distance = []
    for i in range(len(x_train)):
        #đo khoảng cách từ điểm cần dự đoán đến từng điểm trong tập huấn luyện
        euclid_dist = euclid_distance(x_train[i],x_test)
        #lưu các cặp giá trị (khoảng cách, labels của điểm đó) vào danh sahc1 distance
        distance.append((euclid_dist,y_train[i]))
    # sắp xếp theo khoảng cách tăng dần rồi lấy k hàng xóm gần nhất, x[0] ở đây là lấy phần tử đầu (khoảng cách) trong danh sách distance
    neighbor = sorted(distance,key=lambda x: x[0])[:k] 
    # lấy danh sách labels trong k hàng xóm
    label = [label for _, label in neighbor]
    #Tìm ra label xuất hiện nhiều nhất most_common(1) [0] lấy ra phần tử đầu tiên của list [0] lấy ra tên nhãn
    prediction = Counter(label).most_common(1)[0][0]
    return prediction
    

# Chạy mô hình

## Kiểm mô hình trên tập test

In [32]:
def acur_score(y_true, y_pred):
    #So sánh đáp án với máy đoán. Cộng các giá trị mà máy đoán = với đáp án sau đó chia cho tổng để tìm ra tỷ lệ
    return np.sum(y_true == y_pred) / len(y_true)

k = 5
prediction = []

for i in range(len(x_test)):
    # In tiến độ mỗi khi i % 100 để biết code vẫn đang chạy. 
    # #\r: lệnh cho con trỏ chuột không xuống dòng mới (in đè lên dòng cũ) 
    # #end="": ép không cho xuống dòng sau khi in xong
    if i % 100 == 0:
        print(f"\rTiến độ: {i}/{len(x_test)} ({i/len(x_test)*100:.1f}%)", end="")
        
    pred = knn(x_train, x_test[i], y_train, k)
    prediction.append(pred)

print("\nHoàn thành!")
acc = acur_score(np.array(y_test),np.array(prediction))

print("====Kết quả mô hình KNN====")
print("Với k = ",k)
print("Accuracy = ", acc)

Tiến độ: 3900/3999 (97.5%)
Hoàn thành!
====Kết quả mô hình KNN====
Với k =  5
Accuracy =  0.9559889972493123


Note: tốn khoảng 9p để chạy KNN vì dataset có tới 20000 dữ liệu